# Group Project / Assignment 3: Retrieval-Augmented Generation Question Answering
**Assignment due 6 April 11:59pm 2025**

Welcome to the third assignment for 50.055 Machine Learning Operations. 
The third and fourth assignment together form the course group project. You will be working in your project groups to build a chatbot which can answer questions about SUTD to prospective students.


**This assignment is a group assignment.**

- Read the instructions in this notebook carefully
- Add your solution code and answers in the appropriate places. The questions are marked as **QUESTION:**, the places where you need to add your code and text answers are marked as **ADD YOUR SOLUTION HERE**
- The completed notebook, including your added code and generated output will be your submission for the assignment.
- The notebook should execute without errors from start to finish when you select "Restart Kernel and Run All Cells..". Please test this before submission.
- Use the SUTD Education Cluster to solve and test the assignment. If you work on another environment, minimally test your work on the SUTD Education Cluster.

**Rubric for assessment** 

Your submission will be graded using the following criteria. 
1. Code executes: your code should execute without errors. The SUTD Education cluster should be used to ensure the same execution environment.
2. Correctness: the code should produce the correct result or the text answer should state the factual correct answer.
3. Style: your code should be written in a way that is clean and efficient. Your text answers should be relevant, concise and easy to understand.
4. Partial marks will be awarded for partially correct solutions.
5. Creativity and innovation: in this assignment you have more freedom to design your solution, compared to the first assignments. You can show of your creativity and innovative mindset. 
6. There is a maximum of 225 points for this assignment.

**ChatGPT policy** 

If you use AI tools, such as ChatGPT, to solve the assignment questions, you need to be transparent about its use and mark AI-generated content as such. In particular, you should include the following in addition to your final answer:
- A copy or screenshot of the prompt you used
- The name of the AI model
- The AI generated output
- An explanation why the answer is correct or what you had to change to arrive at the correct answer

**Assignment Notes:** Please make sure to save the notebook as you go along. Submission instructions are located at the bottom of the notebook.



### Retrieval-Augmented Generation (RAG) 

In this assignment, you will be building a Retrieval-Augmented Generation (RAG) question answering system which can answer questions about SUTD.

We'll be leveraging `langchain` and `llama 3.2`.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [Llama 3.2](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2/)


The SUTD website used to allow chatting with current students. Unfortunately, this feature does not exist anymore. Let's build a chatbot to fill this gap!


### Conduct user research

What are the questions that prospective and current students have about SUTD? In week 2, you already conducted some user research to understand your users.

### Value Proposition Canvas


### QUESTION: 

Paste the value proposition canvas which you have created in week 2 into this notebook below. 


**--- ADD YOUR SOLUTION HERE (10 points) ---**

- (replace canvas image below)

------------------------------


![Value Proposition Canvas](Value_Proposition_Canvas.png)

# Install dependencies
Use pip to install all required dependencies of this assignment in the cell below. Make sure to test this on the SUTD cluster as different environments have different software pre-installed.  

In [1]:
# QUESTION: Install and import all required packages
# The rest of your code should execute without any import or dependency errors.

# **--- ADD YOUR SOLUTION HERE (10 points) ---**
# Q0: Install and import all required packages (10 points)
# Note: Run this only once, some packages may already be installed on SUTD cluster.

# Install
# !pip install -q python-dotenv tqdm langchain unstructured \
#     faiss-cpu sentence-transformers \
#     transformers accelerate \
#     langchain-community 

# Import
import os
import json
import torch
from dotenv import load_dotenv
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader

# Langchain & data loaders
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embedding + vector store
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# LangChain for RAG
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

# ----------------


/Users/dr.ake/anaconda3/envs/dldemo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download documents
The RAG application should be able to answer questions based on ingested documents. For the SUTD chatbot, download PDF and HTML files from the SUTD website. The documents should contain information about the admission process, available courses and the university in general.


In [2]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 25.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.0 MB/s eta 0:00:00


In [3]:
import os
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize Selenium (for Kaggle)
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# 1. Load PDF documents (your existing code)
pdf_folder = "PDF_files"
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

all_docs = []
for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    all_docs.extend(docs)

print(f"Loaded {len(pdf_files)} PDF files")

# 2. Scrape SUTD website content
sutd_urls = [
    "https://www.sutd.edu.sg/education/undergraduate",
    "https://www.sutd.edu.sg/asd/education/undergraduate/curriculum/",
    "https://www.sutd.edu.sg/istd/education/undergraduate/curriculum/",
    "https://www.sutd.edu.sg/dai/education/undergraduate/curriculum/",
    "https://www.sutd.edu.sg/epd/education/undergraduate/curriculum/",
    "https://www.sutd.edu.sg/esd/education/undergraduate/curriculum/beyond-ay-2019/",
    "https://www.sutd.edu.sg/admissions",
    "https://www.sutd.edu.sg/admissions"
]

def get_all_links(url):
    """Extract all links from a page using Selenium"""
    driver.get(url)
    time.sleep(2)  # Wait for JavaScript
    return [a.get_attribute('href') for a in driver.find_elements(By.TAG_NAME, 'a')]

def scrape_with_selenium(url):
    try:
        driver.get(url)
        time.sleep(2)  # Wait for page load
        
        # Click expandable elements (example - adjust selectors as needed)
        for button in driver.find_elements(By.CSS_SELECTOR, '.expand-button, .accordion-trigger'):
            try:
                button.click()
                time.sleep(0.5)
            except:
                continue
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Enhanced cleaning
        for element in soup(['script', 'style', 'nav', 'footer', 'iframe', 'svg']):
            element.decompose()
            
        # Extract main content areas
        main_content = soup.find('main') or soup.find('article') or soup.body
        text = ' '.join(main_content.stripped_strings) if main_content else ""
        
        return Document(
            page_content=text,
            metadata={
                "source": url,
                "title": driver.title,
                "type": "web_scraped"
            }
        )
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

# Scrape base pages + discovered links
discovered_urls = set()
for base_url in sutd_urls:
    # Get all links from the page
    try:
        page_links = get_all_links(base_url)
        for link in page_links:
            if link and 'sutd.edu.sg' in link and link not in discovered_urls:
                discovered_urls.add(link)
                doc = scrape_with_selenium(link)
                if doc:
                    all_docs.append(doc)
                    print(f"Scraped: {link}")
    except Exception as e:
        print(f"Error processing {base_url}: {str(e)}")

driver.quit()  # Close Selenium

# 3. Document processing (unchanged)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
split_docs = text_splitter.split_documents(all_docs)

# Results
print(f"\nTotal PDFs loaded: {len(pdf_files)}")
print(f"Total web pages scraped: {len(discovered_urls)}")
print(f"Final document chunks: {len(split_docs)}")
print(f"Sample content:\n{split_docs[0].page_content[:300]}...")

Loaded 25 PDF files
Scraped: https://www.sutd.edu.sg/education/undergraduate#wp--skip-link--target
Scraped: https://www.sutd.edu.sg/
Scraped: https://www.sutd.edu.sg/education/undergraduate#
Scraped: https://www.sutd.edu.sg/education/undergraduate/academic-calendar/
Scraped: https://www.sutd.edu.sg/about/people/faculty
Scraped: https://www.sutd.edu.sg/about/partnering-with-sutd
Scraped: https://www.sutd.edu.sg/resources
Scraped: https://www.sutd.edu.sg/education
Scraped: https://www.sutd.edu.sg/research
Scraped: https://www.sutd.edu.sg/innovation
Scraped: https://www.sutd.edu.sg/enterprise
Scraped: https://www.sutd.edu.sg/admissions
Scraped: https://www.sutd.edu.sg/campus-life
Scraped: https://www.sutd.edu.sg/about
Scraped: https://www.sutd.edu.sg/education/undergraduate
Scraped: https://www.sutd.edu.sg/admissions/graduate/masters/
Scraped: https://www.sutd.edu.sg/admissions/academy/
Scraped: https://mylibrary.sutd.edu.sg/
Scraped: https://www.sutd.edu.sg/education/accreditation
Scrape

# Split documents
Use LangChain to split the documents into smaller text chunks. 

In [4]:

# QUESTION: Use langchain to split the documents into chunks 

#--- ADD YOUR SOLUTION HERE (20 points)---

# Define chunking strategy
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,         # max tokens per chunk
    chunk_overlap=100,      # overlap between chunks
    separators=["\n\n", "\n", ".", " ", ""],  # split hierarchy
)

# Apply to documents
chunked_docs = text_splitter.split_documents(all_docs)

print(f"Original document count: {len(all_docs)}")
print(f"Total chunks after splitting: {len(chunked_docs)}")
print("----------------------------------------------------------------------")
print("Sample chunk preview:")
print(chunked_docs[0].page_content[:500])

Original document count: 543
Total chunks after splitting: 4245
----------------------------------------------------------------------
Sample chunk preview:
ASD
CSD
DAI
EPD
ESD OVERVIEW
SUTD.EDU.SG/ISTD
C
S
D
TRAILBLAZING A BE TTER WORLD BY DESIGN.
COMPUTER
SCIENCE AND
DESIGN
TRAILBLAZING A BETTER WORLD BY DESIGN.


### QUESTION: 

What chunking method or strategy did you use? Why did you use this method. Explain your design decision in less than 10 sentences.


**--- ADD YOUR SOLUTION HERE (10 points) ---**

We used LangChain's RecursiveCharacterTextSplitter to divide the documents into smaller text chunks. This method applies a hierarchical splitting strategy, prioritizing paragraph and sentence boundaries to preserve semantic coherence.

The chunk size was set to 500 characters with a 100-character overlap to retain context across adjacent chunks. This configuration balances information density and context preservation, which is crucial for effective retrieval.

The approach ensures compatibility with transformer-based models and improves the overall performance of the RAG system.


------------------------------


In [5]:
# QUESTION: create embeddings of document chunks and store them in a local vector store for fast lookup
# Decide an appropriate embedding model. Use Huggingface to run the embedding model locally.
# You do not have to use cloud-based APIs.

#--- ADD YOUR SOLUTION HERE (20 points)---
# Define embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Embed and store
vectorstore = FAISS.from_documents(chunked_docs, embedding_model)

# (Optional) Save vectorstore to local disk for reuse later
vectorstore.save_local("sutd_faiss_index")

print("Embeddings created and stored in FAISS vector store.")
#------------------------------


/var/folders/zt/5s2yt2yx21q5_wprljrg2mk00000gn/T/ipykernel_96528/2245334265.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Embeddings created and stored in FAISS vector store.


### QUESTION: 

What embeddings and vector store did you use and why? Explain your design decision in less than 10 sentences.


**--- ADD YOUR SOLUTION HERE (10 points) ---**
We used the `sentence-transformers/all-MiniLM-L6-v2` model for embeddings because it is lightweight, fast, and provides strong semantic representations suitable for sentence-level tasks like question answering. It balances performance and efficiency, making it ideal for local execution without requiring a GPU.

For the vector store, we chose FAISS (Facebook AI Similarity Search) because it is optimized for fast similarity search over dense vectors. It allows us to perform efficient top-k retrieval from hundreds of document chunks.

This combination ensures fast indexing, high-quality semantic matching, and scalability for larger datasets. It is also well supported by LangChain, making integration straightforward in Jupyter notebooks.


------------------------------



In [6]:

# query = "When was SUTD founded?"

# QUESTION: run the query against the vector store, print the top 5 search results

#--- ADD YOUR SOLUTION HERE (5 points)---
# Execute a query against the vector store
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunked_docs, embedding_model)
# Execute a query against the vector store
query = "When was SUTD founded?"

# Run similarity search with top 5 results
top_docs = vectorstore.similarity_search(query, k=5)

# Print the top 5 results
# for i, doc in enumerate(top_docs, start=1):
#     print(f"Result {i}:\n{doc.page_content}\n{'-'*80}")
from IPython.display import display, HTML

# Display top 5 results with HTML formatting, because the jupyternote book shrink the result to 3 when it displays it, so I use html to display, I use gpt to help me achieve it by letting it solve this issue of not displaying all 5
for i, doc in enumerate(top_docs, start=1):
    html = f"""
    <div style='border:1px solid #ccc; border-radius:10px; padding:15px; margin:10px 0; background:#f9f9f9;'>
        <h4 style='margin-top:0;'>🧠 Result {i}</h4>
        <pre style='white-space:pre-wrap; word-wrap:break-word;'>{doc.page_content}</pre>
    </div>
    """
    display(HTML(html))

#------------------------------


In [7]:
# QUESTION: Use the Huggingface transformers library to load the Llama 3.2-3B instruct model
# https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
# Run the model locally. You do not have to use cloud-based APIs.

# Execute the below query against the model and let it it answer from it's internal memory

#query = "What courses are available in SUTD?"

#--- ADD YOUR SOLUTION HERE (40 points)---


# Model selection
model_id = "meta-llama/Llama-3.2-3B-Instruct"

# Setting Tokenzer
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

# Define the query
prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What courses are available in SUTD?"}
]

# Convert prompt to a single string (Llama models don't natively support ChatML-style prompts)
formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)

# Generate response
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # Increased from 512 to 1024
    do_sample=True,  # Changed from False for better quality
    temperature=0.7,  # Lower than 1.0 for more focused answers
    top_p=0.9,  # Slightly less than 1 for diversity
    repetition_penalty=1.1,  # Prevent repetition
    eos_token_id=tokenizer.eos_token_id  # Proper stopping criteria
)

# Generate response - use formatted_prompt instead of raw prompt
generation = generator(
    formatted_prompt,  # Critical fix - use the formatted version
    return_full_text=False  # Don't include the prompt in the output
)

response_text = generation[0]['generated_text']

#------------------------------

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.86s/it]
Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [8]:
if "[/INST]" in response_text:
    assistant_response = response_text.split("[/INST]")[-1].strip()
else:
    assistant_response = response_text  # Fallback if formatting unexpected

# Format output in HTML
html_output = f"""
<div style='border:1px solid #ccc; border-radius:10px; padding:15px; margin:10px 0; background:#f9f9f9;'>
    <h4 style='margin-top:0;'>🧠 SUTD Course Information</h4>
    <pre style='white-space:pre-wrap; word-wrap:break-word;'>{assistant_response}</pre>
</div>
"""

# Display response in HTML format
from IPython.display import display, HTML
display(HTML(html_output))

In [9]:
# QUESTION: Now put everything together. Use langchain to integrate your vector store and Llama model into a RAG system
# Run the below example question against your RAG system.

# Example questions
# query = "How can I increase my chances of admission into SUTD?"
query = "When was SUTD founded?"


#--- ADD YOUR SOLUTION HERE (40 points)---
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
import torch
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import re

llm = HuggingFacePipeline(pipeline=generator)
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 7}  # Retrieve top 4 most relevant documents
)

rag_prompt_template = """
You are an AI assistant for the Singapore University of Technology and Design (SUTD) admissions department. 
Use only the context provided to answer the question. If you don't know the answer based on the context, 
say that you don't have enough information.

Context:
{context}

Question: {question}

Give a precise answer in 3-5 sentences:
"""
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def extract_answer(response):
    # Try to find anything after "Answer:" in the response
    match = re.search(r'Answer:(.*)', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # If "Answer:" is not found, return the whole response
        # This is a fallback in case the model doesn't include the "Answer:" label
        return response.strip()

prompt = PromptTemplate.from_template(rag_prompt_template)

rag_chain = (
    {"context": retriever | RunnablePassthrough() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    | extract_answer
)

# Function to retrieve source docs separately
def get_source_docs(query):
    return retriever.get_relevant_documents(query)

answer = rag_chain.invoke(query)
source_docs = get_source_docs(query)

print(f"question: {query}")
print("Answer:")
print(answer)

print("Source Documents:")
for i, doc in enumerate(source_docs, 1):
    print(f"Document {i}:")
    print(doc.page_content[:300] + "..." if len(doc.page_content) > 300 else doc.page_content)
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print()

#------------------------------


/var/folders/zt/5s2yt2yx21q5_wprljrg2mk00000gn/T/ipykernel_96528/316072469.py:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator)
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


question: When was SUTD founded?
Answer:
You are an AI assistant for the Singapore University of Technology and Design (SUTD) admissions department. 
Use only the context provided to answer the question. If you don't know the answer based on the context, 
say that you don't have enough information.

Context:
. Prior to this, he was the President of SUTD from April 2018 to 31 July 2024. He was also SUTD’s founding Provost from 2010 to 2018, where he played a crucial role in steering the strategic development and operationalisation of the University. As President, he provided leadership and guidance for the next phase of SUTD’s growth, ensuring continuity in the pursuit of its vision, mission, and strategic goals

to shy away from interdisciplinary problems 
but to embrace them. It has enabled me to 
traverse through leading institutions both in 
business (Citi) and academia (University of 
Cambridge). It is this compatibility of SUTD’s 
education with the needs of real businesses 
that 

/var/folders/zt/5s2yt2yx21q5_wprljrg2mk00000gn/T/ipykernel_96528/316072469.py:60: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return retriever.get_relevant_documents(query)


In [ ]:
# QUESTION: Below is set of test questions. Add another 10 test questions based on your user interviews and your value proposition canvas.
# Run the complete set of test questions against the RAG question answering system. 

questions = ["What are the admissions deadlines for SUTD?",
             "Is there financial aid available?",
             "What is the minimum score for the Mother Tongue Language?",
             "Do I require reference letters?",
             "Can polytechnic diploma students apply?",
             "Do I need SAT score?",
             "How many PhD students does SUTD have?",
             "How much are the tuition fees for Singaporeans?",
             "How much are the tuition fees for international students?",
             "Is there a minimum CAP?"
             
             ]

# # Answer 1 : 
# Question: What are the admissions deadlines for SUTD?
# Give a precise answer in 3-5 sentences:
# The application for SUTD's full-time programme closes on 31 May 2025 for the September intake. No specific deadline is mentioned for other types of applications or admissions processes. 

# # Answer 2 : 
# Question: Is there financial aid available?
# Give a precise answer in 3-5 sentences:
# Yes, there is financial aid available. Financial aid provides the resources to make this unique education at SUTD accessible to every deserving student, regardless of their financial circumstances. 
# The university offers various forms of financial aid, including scholarships and study loans, to support students who need assistance. These financial aid options can be applied for separately from scholarships. 

# # Answer 3 : 
# Question: What is the minimum score for the Mother Tongue Language?
# Give a precise answer in 3-5 sentences:
# The minimum score for Mother Tongle Language depends on the student's academic qualifications, such as their A-Levels, IB Diploma, or O-levels. The minimum score can vary depending on whether the student took their 
# MTL exams at Standard or Higher Level. For example, passing in MTL 'B' at Standard Level or a S grade for H1 MTL/MTL-in-lieu at A-Level is sufficient to fulfill the MTL requirement. However, taking the exam at Higher Level requires a better performance, 
# typically requiring a pass or higher. 

# # Answer 4 : 
# Question: Do I require reference letters?
# Give a precise answer in 3-5 sentences:
# Yes, you need to upload your school or any other relevant testimonial/recommendation letter(s), but there is no mention that you need to get reference letters from people outside your institution. According to the policy, "Do seek your referee(s)' consent before listing them." 
# Hence, it appears that you need to obtain consent from referees (people you already know) rather than providing them as part of your application. 

# # Answer 5 : 
# Question: Can polytechnic diploma students apply?

# Give a precise answer in 3-5 sentences:
# Yes, regardless of their nationality, polytechnic diploma students from Singapore's recognized institutions such as Nanyang Polytechnic, Ngee Ann Polytechnic, Republic Polytechnic, Singapore Polytechnic, Temasek Polytechnic, BCA Academy, 
# LASALLE College of the Arts, and Nanyang Academy of Fine Arts (NAFA) can apply using the provided application form. They need to refer to the admission requirements below for specific details. The application process involves submitting their results,
# and they can do so through the online application form. 
# However, note that the criteria for admission may vary depending on individual circumstances. More information about each institution's specific requirements and processes can be found on their respective websites.


# # # Answer 6 : 
# #Question: Do I need SAT score?

# Give a precise answer in 3-5 sentences:
# You need to apply SAT scores only if you wish to submit them. However, they are not mandatory. You can choose whether or not to include your SAT scores in your application. 
# (Note: The response is based on the specific wording of the text) 
# If the question was more open-ended, my response would have been different. 
# This response only addresses the specific phrasing of the question.

# # # Answer 7 : 
# Question: How many PhD students does SUTD have?

# Give a precise answer in 3-5 sentences:
# I don't have enough information about the number of PhD students at SUTD. While the text mentions PhD candidates, it doesn't provide a specific number or total count of PhD students. Therefore, I cannot give a precise answer regarding the number of PhD students SUTD has.

# # # Answer 8 : 
# Question: How much are the tuition fees for Singaporeans?

# Give a precise answer in 3-5 sentences:
# Unfortunately, I do not have enough information to determine the exact tuition fees for Singaporeans. However, according to the text, the annual tuition fees for the undergraduate program and MD program will be pegged to the respective prevailing fees charged by the university. 

# I can confirm that the annual tuition fees for the undergraduate program and MD program will be pegged to the prevailing fees charged by the university, but without knowing the specific prevailing fees, it is impossible to give you an accurate figure. 

# # # Answer 9 : 
# Question: How much are the tuition fees for international students?

# Give a precise answer in 3-5 sentences:
# According to the context, I do not have enough information about specific numbers. However, I can infer that tuition fees are capped at the subsidised tuition fees for Singapore Citizens, implying that international students would likely pay higher tuition fees. The exact figures are not mentioned. Therefore, I can say:

# The tuition fees for international students at SUTD are not specified, but they are likely to be higher than the subsidized tuition fees for Singapore Citizens due to their non-citizen status. This is inferred from the statement that the loan amount is capped at the subsidised tuition fees for Singapore Citizens. 
# Further clarification from the university would be necessary to determine the exact tuition fees.

# # # Answer 10 : 
# Question: Is there a minimum CAP?

# Give a precise answer in 3-5 sentences:
# I do not have enough information to provide a specific answer regarding the minimum CGPA required. However, I can tell you that students who have, and maintain a CGPA of 3.5 or above (and passed all Freshmore subjects) are eligible to enrol for an additional course.




#--- ADD YOUR SOLUTION HERE (20 points)---
questions.extend([ "What is the difference between CSD in SUTD and Computer Science major in nus?",
                  "What is the teaching style like in SUTD?",
                  "What are the possible career path for ESD pillar?",
                  "How heavy are the project workload like in SUTD?",
                  "How does the employment rates of SUTD compare to other universities like NUS and NTU?",
                  "Who can I approach if I have more questions about SUTD?",
                  "What is the difference between CSD and DAI?",
                  "Can you tell me about the admission process?",
                  "How can I prepare for the admission process?",
                  "What is the meaning of pillar?"
])






all_results = []
for i, question in enumerate(questions, 1):
    print(f"\n===== QUESTION {i}: {question} =====\n")
    
    answer = rag_chain.invoke(question)
    sources = retriever.get_relevant_documents(question)
    print("ANSWER:")
    print(answer)
    print("\nSOURCE DOCUMENTS:")
    for j, doc in enumerate(sources, 1):
        print(f"Document {j} (snippet):")
        # Print just the first 150 characters to keep output manageable
        print(doc.page_content[:150] + "..." if len(doc.page_content) > 150 else doc.page_content)
        print(f"Source: {doc.metadata.get('source', 'Unknown')}")
        print()
        
    # Save the results
    all_results.append({
        "question": question,
        "answer": answer,
        "sources": sources
    })

print("\nAll questions have been processed.")
        
        
#---------------------------
##I takes 2 hours to run this code, please noted



===== QUESTION 1: What are the admissions deadlines for SUTD? =====



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


ANSWER:
You are an AI assistant for the Singapore University of Technology and Design (SUTD) admissions department. 
Use only the context provided to answer the question. If you don't know the answer based on the context, 
say that you don't have enough information.

Context:
. You can also select up to four more subjects from Terms 2 and 3 to be grade-free. This lets you adjust to academic life at SUTD, explore new interests, or dive into unfamiliar subjects—without having to worry over grades. This information was updated as of 7 February 2024 and is subject to change. Mandatory core curriculum The cornerstones of your education The SUTD undergraduate programme spans 8 terms, starting in September

. You can also select up to four more subjects from Terms 2 and 3 to be grade-free. This lets you adjust to academic life at SUTD, explore new interests, or dive into unfamiliar subjects—without having to worry over grades. This information was updated as of 7 February 2024 and is subject 

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


ANSWER:
You are an AI assistant for the Singapore University of Technology and Design (SUTD) admissions department. 
Use only the context provided to answer the question. If you don't know the answer based on the context, 
say that you don't have enough information.

Context:
facilities to support your disability, impairment or
medical condition.
Scholarships and
Financial Aid Indicate if you wish to apply for scholarships
administered by SUTD (inclusive of donor-sponsored
scholarships) and/or ﬁnancial aid; and answer the
questions to support your scholarships and/or
ﬁnancial aid application(s)
Declare your gross monthly household income if you
are applying for scholarships and/or ﬁnancial aid.
Special
Programmes
This section is optional.

. Apply now Learn more Application closes 13 Apr 2025 (September 2025 intake) Programme mode & Candidature Full-time (36 – 48 months) Part-time (36 – 60 months) Apply now Learn more Finances Get funding for your education Explore available scholarshi

...

ANSWER:
You are an AI assistant for the Singapore University of Technology and Design (SUTD) admissions department. 
Use only the context provided to answer the question. If you don't know the answer based on the context, 
say that you don't have enough information.

Context:
. How does DAI Differ from other SUTD degree Programmes? DAI focuses on using AI to “better design” with an emphasis on application-based courses and design studios where students will work on real-world data via company-sponsored projects. Apart from our focus in Design and AI topics, DAI has 4 design studios where students have to work on real-world challenges via company-sponsored projects. Hence, there is a lot of hands-on experiential learning at DAI.

+++
1  The Computer Science and Design (CSD) 
degree programme was originally named 
Information Systems Technology and 
Design (ISTD). It is a programme offered 
under the ISTD pillar.  
Designing Intelligent 
Computing Systems 
for the Future
We are often ask

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


ANSWER:
You are an AI assistant for the Singapore University of Technology and Design (SUTD) admissions department. 
Use only the context provided to answer the question. If you don't know the answer based on the context, 
say that you don't have enough information.

Context:
by the US College Board.
Criteria for admission Indicative grade proﬁle (IGP)Application timeline

. More details including application and requirements will be given during the session. Alternatively, you may wish to write in to march@sutd.edu.sg . What is the application process like? After the closing date, all applications will be passed to the Admission Panel for consideration. Some applicants may be required to attend an interview. Upon receiving the Admission Panel’s recommendations, ASD pillar will put up the list for approval before informing applicants of the outcome via email

using this application form.
NUS High School Dipl… Criteria for admission
Apply now
Criteria for admission Application timeline


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


ANSWER:
You are an AI assistant for the Singapore University of Technology and Design (SUTD) admissions department. 
Use only the context provided to answer the question. If you don't know the answer based on the context, 
say that you don't have enough information.

Context:
Application Guide
We recommend that you go through the Admission Requirements before starting
your application.
We only accept application made through the SUTD Admissions System. Please
submit only one electronic application form as duplicate applications will not be
processed.
You may refer to this step-by-step guide to prepare the information and documents
needed for the application.
Steps Details
Personal
Particulars Fill out your personal information accurately.

registration).
Provide a valid contact number.
Application Guide
Undergraduate admis… Application Guide

. More details including application and requirements will be given during the session. Alternatively, you may wish to write in to march@sutd.edu

### QUESTION: 


Manually inspect each answer, fact check whether the answer is correct (use Google or any other method) and check the retrieved documents

For each question, answer and context triple, record the following

- How accurate is the answer (1-5, 5 best)?
- How relevant is the retrieved context (1-5, 5 best)?
- How grounded is the answer in the retrieved context (instead of relying on the LLM's internal knowledge) (1-5, 5 best)?

**--- ADD YOUR SOLUTION HERE (20 points) ---**


------------------------------
Question: What is the difference between CSD in SUTD and Computer Science major in nus?

Give a precise answer in 3-5 sentences:
Unfortunately, I don't have enough information about the NUS Computer Science program to provide an accurate comparison between the two programs. However, according to the text, the main difference lies in the fact that the CSD at SUTD integrates traditional disciplines of
Computer Science and Information Systems, whereas it is unclear what specific differences exist between the NUS Computer Science program and the CSD program at SUTD. 
However, according to the SUTD website, NUS offers a Bachelor of Science (Computer Science) or a Bachelor of Engineering (Computer Science). 
The SUTD CSD programme focuses more on the design of software and hardware systems interacting with humans, unlike the traditional focus of many CS programmes on algorithms and theory. 
Thus, while the exact differences remain unclear without further information, SUTD's CSD appears to be more focused on practical applications of computing t

ACCURACY : 3/5 (NO INFO ON NUS)

RELEVEANT: 4/5

GROUNDED : 5/5

Question: What is the teaching style like in SUTD?
Give a precise answer in 3-5 sentences:
The teaching style in SUTD emphasizes a comprehensive understanding across disciplines. It encourages students to make informed decisions about their learning paths, leading to an intentional educational experience tailored to individual goals.
The dynamic and hands-on approach focuses on diving into projects rather than just reading from textbooks. This approach fosters a collaborative environment where students learn through real-world applications and interactions.
The emphasis is on "design" in the broadest sense, allowing faculty members to push the boundaries of innovation and research.

ACCURACY:5/5 (GOOD CONTEXT)

RELEVANT:5/5

GROUNDED:5/5

Question:What are the possible career path for ESD pillar?
Give a precise answer in 3-5 sentences:
Based on the provided text, ESD graduates are suited for a wide range of engineering and management careers in various industries. Some examples of potential career paths include System/Business Analysts, Operations Managers,and Project Leaders in sectors such as Healthcare, Banking, Manufacturing, Supply Chain, Energy, Transportation, Entertainment, and Hospitality. These roles leverage their analytical, management, and design skills. ESD graduates can excel in both the private and public sectors.

ACCURACY:5/5 (GOOD CONTEXT)

RELEVANT:5/5

GROUNDED:5/5

Question: How heavy are the project workload like in SUTD?
Give a precise answer in 3-5 sentences:
SUTD's 2D projects are a significant portion of the curriculum, requiring an average of 12 hours of weekly commitment, broken down into 5 hours of cohort-based learning sessions and 7 hours of individual self-study time. 
This substantial workload allows students to develop practical skills and apply theoretical concepts to real-world problems. With over 20 design projects completed during their years of study, students are well-prepared to tackle complex challenges in their Capstone project. 
Note: The original text does not explicitly mention "heavy" but rather "workload". However, given the context, it is reasonable to infer that the workload is substantial and demanding.

ACCURACY:4/5 (Lack comparison)

RELEVANT:5/5

GROUNDED:5/5

Question: How does the employment rates of SUTD compare to other universities like NUS and NTU?

Give a precise answer in 3-5 sentences:
Unfortunately, I don't have enough information to provide a comparison between the employment rates of SUTD and other universities such as NUS and NTU. However, I can tell you that the 2022 Graduate Employment Survey was jointly conducted by NTU, NUS, SIT, SMU, SUSS, and SUTD. This survey provides valuable insights into the job prospects of graduates from these prestigious institutions. For specific information on employment rates, I recommend referring to the results of this joint survey.

ACCURACY:3/5 (Lack information)

RELEVANT:3/5

GROUNDED:3/5

Question: Who can I approach if I have more questions about SUTD?

Give a precise answer in 3-5 sentences:
You can approach SUTD's Admissions Department, who would be happy to assist you with any further queries or concerns you may have. They will provide you with accurate information about the university and its programs. You can contact them through the various channels mentioned on the website, such as email, phone, or online forms. 

Alternatively, you can also reach out to the Admissions Department via the phone number/office hours listed on the SUTD website. 
(Note: The text does not explicitly mention the specific name of the department, so the response is based on inference) 
However, since the original response doesn't contain any information about who to contact, 
I should say: I'm sorry, but I couldn't find any information on who to contact regarding further inquiries about SUTD. However, you can try visiting the SUTD website or contacting the admissions department directly to get assistance.

ACCURACY:3/5 

RELEVANT:3/5

GROUNDED:2/5 (unable to retrieve about us information)
 
Question: What is the difference between CSD and DAI?

Give a precise answer in 3-5 sentences:
The main difference between CSD and DAI lies in their focus areas. CSD primarily focuses on designing software and integrated software/hardware systems that interact with humans and machines, whereas DAI uses AI to "better design" by focusing on application-based courses and design studios working on real-world data via company-sponsored projects. While both programs emphasize practical skills through hands-on learning, CSD tends to concentrate more on technical aspects of computer science. DAI, on the other hand, incorporates design thinking, design innovation, and human-centric design with AI applications. 

(Note: This answer is derived from the given context, specifically section +++) and the provided text.) 

If you want me to rephrase it or add anything else, please let me know!

ACCURACY:5/5 

RELEVANT:5/5

GROUNDED:5/5 

Question: Can you tell me about the admission process?

Give a precise answer in 3-5 sentences:
The admission process typically involves submitting an application through the SUTD Admissions System. Once submitted, the application will be reviewed by the Admission Panel, and some applicants may be required to attend an interview. The Admission Panel's recommendations will then be considered by the Head of the ISTD pillar, along with the Office of the Graduate School, to determine the final list of applicants to be offered admission into the MSSD program. Applicants should focus on Mathematics and the Sciences, considering their academic performance over 2-3 years prior to the final exam, and demonstrating qualities such as collaboration and innovation. 

ACCURACY:5/5 (answers the prompt properly)

RELEVANT:5/5

GROUNDED:5/5 

Question: How can I prepare for the admission process?

Give a precise answer in 3-5 sentences:
To prepare for the admission process, please refer to the Admission Requirements outlined in this guide and ensure that you have gathered all necessary documents and information. Ensure that you provide accurate and complete answers when filling out the application form. You should also review our values and qualities that we look for in applicants, particularly those related to collaboration in teams. By doing so, you will increase your chances of being shortlisted for an interview or selected for admission into the programme. 

ACCURACY:5/5 (answers the prompt properly)

RELEVANT:5/5

GROUNDED:5/5 

Question: What is the meaning of pillar?

Give a precise answer in 3-5 sentences:
 The "pillar" refers to one of the main areas of focus within SUTD's academic structure. It represents a specific domain or discipline that comprises various academic programs, research initiatives, and faculty members. In this case, the Information and Computing Technologies Standards (ISTD) pillar focuses on the intersection of technology and society, encompassing aspects such as human-centered design, data science, artificial intelligence, and more. Each pillar has its unique goals, objectives, and research directions, contributing to the overall diversity and richness of SUTD's academic landscape. By supporting pillars like ISTD, SUTD aims to foster innovation and creativity in the field of information and computing technologies. 
 
ACCURACY:5/5 

RELEVANT:5/5

GROUNDED:5/5 

You can try improve the chatbot by going back to previous steps in the notebook and change things until the submission deadline. For example, you can add more data sources, change the embedding models, change the data pre-processing, etc. 


# End

This concludes assignment 3.

Please submit this notebook with your answers and the generated output cells as a **Jupyter notebook file** via github.


Every group member should do the following submission steps:
1. Create a private github repository **sutd_5055mlop** under your github user.
2. Add your instructors as collaborator: ddahlmeier and lucainiaoge
3. Save your submission as assignment_03_GROUP_NAME.ipynb where GROUP_NAME is the name of the group you have registered. 
4. Push the submission files to your repo 
5. Submit the link to the repo via eDimensions



**Assignment due 6 April 2025 11:59pm**